In [2]:
# try:
#   import apex
# except Exception:
#   ! git clone https://github.com/NVIDIA/apex.git
#   ! cd apex
#   ! pip install --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" /content/apex

In [3]:
# !pip install nemo_toolkit['all']

In [4]:
# !apt-get update && apt-get install -y libsndfile1 ffmpeg
# !git clone https://github.com/NVIDIA/NeMo
# %cd NeMo
# !bash reinstall.sh

In [1]:
BRANCH = 'main'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[nlp]

DEPRECATION: git+https://github.com/NVIDIA/NeMo.git@main#egg=nemo_toolkit[nlp] contains an egg fragment with a non-PEP 508 name pip 25.0 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/11617
  Cloning https://github.com/NVIDIA/NeMo.git (to revision main) to /tmp/pip-install-klsbzhvy/nemo-toolkit_a575650814fe4abc929c35c02f438b79
  Running command git clone --filter=blob:none --quiet https://github.com/NVIDIA/NeMo.git /tmp/pip-install-klsbzhvy/nemo-toolkit_a575650814fe4abc929c35c02f438b79
  Resolved https://github.com/NVIDIA/NeMo.git to commit fd9329da37460cd7df167c45147dd35c0772bba0
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.8/117.8 kB 

In [5]:
! pip install ipywidgets
! jupyter nbextension enable --py widgetsnbextension

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 10.7 MB/s eta 0:00:00
Enabling notebook extension jupyter-js-widgets/extension...
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json
Paths used for configuration of notebook: 
    	
      - Validating: OK
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json


In [6]:
!mkdir -p models/mistral7b

In [7]:
import huggingface_hub

huggingface_hub.snapshot_download(repo_id="mistralai/Mistral-7B-v0.1", local_dir="models/mistral7b", local_dir_use_symlinks=False)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 14 files:   0%|          | 0/14 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

...

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/5.06G [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

'/content/models/mistral7b'

In [8]:
!python /content/NeMo/scripts/nlp_language_modeling/convert_hf_mistral_7b_to_nemo.py --in-file=/content/models/mistral7b --out-file=/content/models/mistral7b.nemo

python3: can't open file '/content/NeMo/scripts/nlp_language_modeling/convert_hf_mistral_7b_to_nemo.py': [Errno 2] No such file or directory


In [9]:
!git clone https://github.com/pubmedqa/pubmedqa.git
!cd pubmedqa/preprocess && python split_dataset.py pqal

Cloning into 'pubmedqa'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 40 (delta 0), reused 1 (delta 0), pack-reused 37
Receiving objects: 100% (40/40), 704.87 KiB | 12.37 MiB/s, done.
Resolving deltas: 100% (12/12), done.


In [10]:
import json

def write_jsonl(fname, json_objs):
    with open(fname, 'wt') as f:
        for o in json_objs:
            f.write(json.dumps(o) + '\n')

In [11]:
def from_question(obj):
    st = ""
    st += f"QUESTION: {obj['QUESTION']}\n"
    st += "CONTEXT: "
    for i, label in enumerate(obj["LABELS"]):
        st += f"{obj['CONTEXTS'][i]}\n"

    st += f"TARGET: the answer to the question given the context is (yes|no|maybe): "
    return st

In [12]:
def convert_to_jsonl(data_path, out_path):
    data = json.load(open(data_path, "rt"))
    json_objs = []
    for k in data.keys():
        obj = data[k]
        prompt = from_question(obj)
        completion = obj["reasoning_required_pred"]
        json_objs.append({"input": prompt, "output": completion})

    write_jsonl(out_path, json_objs)
    return json_objs

In [13]:
train_json_objs = convert_to_jsonl("pubmedqa/data/pqal_fold0/train_set.json", "pubmedqa_train.jsonl")
dev_json_objs = convert_to_jsonl("pubmedqa/data/pqal_fold0/dev_set.json", "pubmedqa_val.jsonl")
test_json_objs = convert_to_jsonl("pubmedqa/data/test_set.json", "pubmedqa_test.jsonl")


In [14]:
test_json_objs[0]

{'input': 'QUESTION: Is anorectal endosonography valuable in dyschesia?\nCONTEXT: Dyschesia can be provoked by inappropriate defecation movements. The aim of this prospective study was to demonstrate dysfunction of the anal sphincter and/or the musculus (m.) puborectalis in patients with dyschesia using anorectal endosonography.\nTwenty consecutive patients with a medical history of dyschesia and a control group of 20 healthy subjects underwent linear anorectal endosonography (Toshiba models IUV 5060 and PVL-625 RT). In both groups, the dimensions of the anal sphincter and the m. puborectalis were measured at rest, and during voluntary squeezing and straining. Statistical analysis was performed within and between the two groups.\nThe anal sphincter became paradoxically shorter and/or thicker during straining (versus the resting state) in 85% of patients but in only 35% of control subjects. Changes in sphincter length were statistically significantly different (p<0.01, chi(2) test) in p

In [15]:
!wget https://raw.githubusercontent.com/NVIDIA/NeMo/main/examples/nlp/language_modeling/tuning/conf/megatron_gpt_finetuning_config.yaml

--2024-03-18 22:52:59--  https://raw.githubusercontent.com/NVIDIA/NeMo/main/examples/nlp/language_modeling/tuning/conf/megatron_gpt_finetuning_config.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11392 (11K) [text/plain]
Saving to: ‘megatron_gpt_finetuning_config.yaml’

megatron_gpt_finetu 100%[===================>]  11.12K  --.-KB/s    in 0s      

2024-03-18 22:52:59 (45.6 MB/s) - ‘megatron_gpt_finetuning_config.yaml’ saved [11392/11392]



In [16]:
import hydra
from omegaconf.omegaconf import OmegaConf

hydra.initialize(version_base=None, config_path=".")

hydra.initialize()

In [17]:
cfg = hydra.compose(config_name="megatron_gpt_finetuning_config", overrides=['++model.mcore_gpt=True'])

In [18]:
OmegaConf.update(cfg, "model.data",{
    "train_ds": {
        "num_workers": 0,
        "file_names": ["pubmedqa_train.jsonl"],
        "concat_sampling_probabilities": [1.0]
        },
    "validation_ds": {
        "num_workers": 0,
        "file_names": ["pubmedqa_val.jsonl"]
        },
    "test_ds": {
        "file_names": ["pubmedqa_test.jsonl"],
        "names": ["pubmedqa"]
        }
    }, merge=True)

In [19]:
OmegaConf.update(cfg, "model", {
    "restore_from_path": "models/mistral7b.nemo",
    "peft": {
        "peft_scheme": "lora"
    },
    "tensor_model_parallel_size": 1,
    "pipeline_model_parallel_size": 1,
    "micro_batch_size": 1,
    "global_batch_size": 8,
}, merge=True)

In [20]:
OmegaConf.update(cfg, "trainer", {
    'devices': 1,
    'num_nodes': 1,
    'precision': "bf16-mixed",
    "val_check_interval": 10,
    "max_steps": 20
})

In [21]:
from nemo.collections.nlp.models.language_modeling.megatron_gpt_sft_model import MegatronGPTSFTModel
from nemo.collections.nlp.parts.megatron_trainer_builder import MegatronLMPPTrainerBuilder
from nemo.collections.nlp.parts.peft_config import LoraPEFTConfig
from nemo.utils.exp_manager import exp_manager

trainer = MegatronLMPPTrainerBuilder(cfg).create_trainer()
exp_manager(trainer, cfg.exp_manager)

model_cfg = MegatronGPTSFTModel.merge_cfg_with(cfg.model.restore_from_path, cfg)
model = MegatronGPTSFTModel.restore_from(cfg.model.restore_from_path, model_cfg, trainer=trainer)


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

ImportError: cannot import name 'IndexedDataset' from 'megatron.core.datasets.indexed_dataset' (/usr/local/lib/python3.10/dist-packages/megatron/core/datasets/indexed_dataset.py)

In [ ]:
trainer.test(model)